# Graph7: Agent
- Memory integration

In [3]:
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from openai import OpenAI
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
api_key = os.getenv("open_router_api_key")
model="meta-llama/llama-4-maverick:free"
base_url="https://openrouter.ai/api/v1"

llm = ChatOpenAI(
  openai_api_key=api_key,
  openai_api_base=base_url,
  model_name=model,
)

class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]

C:\Users\maxdb\AppData\Local\Temp\ipykernel_30136\3646470601.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [5]:
def process(state:AgentState) -> AgentState:
    """This node will respond to your input"""
    response = llm.invoke(state['messages'])
    state['messages'].append(AIMessage(content=response.content))
    print(f"\nAI: {response.content}")
    return state

In [ ]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)
agent = graph.compile()

conversation_history = []

user_input = input("Enter Query: ")
while user_input != "exit":
    conversation_history.append(HumanMessage(content=user_input))
    result = agent.invoke({"messages":conversation_history})

    print(result["messages"])
    conversation_history = result["messages"]

    user_input = input("Enter Query: ")

with open("conversation.txt", "w") as f:
    f.write("Your conversation Log:\n")
    for message in conversation_history[-5:]:
        if isinstance(message, HumanMessage):
            f.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            f.write(f"AI: {message.content}\n\n")
    f.write("End of Chat!")

print("Conversation saved to conversation.txt")
